In [2]:
import cv2
import numpy as np
import tensorflow as tf
from huggingface_hub import from_pretrained_keras

2024-08-25 04:15:01.482138: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-08-25 04:15:01.491307: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-25 04:15:01.500875: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-25 04:15:01.504029: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-25 04:15:01.511952: I tensorflow/core/platform/cpu_feature_guar

In [3]:
backbone = from_pretrained_keras("keras-io/monocular-depth-estimation")
live_path = '/home/pessoal/Documents/repos/liveness_system/data/celebA-spoof/CelebA_Spoof_/CelebA_Spoof/Data/test/4930/live/504311.png'
spoof_path = '/home/pessoal/Documents/repos/liveness_system/data/celebA-spoof/CelebA_Spoof_/CelebA_Spoof/Data/test/3613/spoof/541354.png'

Fetching 11 files: 100%|██████████| 11/11 [00:00<00:00, 221814.15it/s]
I0000 00:00:1724570110.191906 1212701 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1724570110.193010 1212701 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1724570110.193104 1212701 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1724570110.194017

In [5]:
# Load the images
live_image = cv2.imread(live_path)
spoof_image = cv2.imread(spoof_path)

# Preprocess the images
live_image = cv2.cvtColor(live_image, cv2.COLOR_BGR2RGB)
spoof_image = cv2.cvtColor(spoof_image, cv2.COLOR_BGR2RGB)
live_image = cv2.resize(live_image, (224, 224))
spoof_image = cv2.resize(spoof_image, (224, 224))
live_image = tf.image.convert_image_dtype(live_image, tf.float32)
spoof_image = tf.image.convert_image_dtype(spoof_image, tf.float32)

live_image = tf.expand_dims(live_image, axis=0)
spoof_image = tf.expand_dims(spoof_image, axis=0)

live_image = (live_image * 2.0) - 1.0


# Make predictions
live_mask = backbone.predict()
spoof_mask = backbone.predict()

live_mask = live_mask[0]
spoof_mask = spoof_mask[0]

print(np.max(live_mask))
print(np.min(live_mask))
print(np.max(spoof_mask))
print(np.min(spoof_mask))

live_mask = (live_mask - np.min(live_mask)) / (np.max(live_mask) - np.min(live_mask)) * 255.0
spoof_mask = (spoof_mask - np.min(spoof_mask)) / (np.max(spoof_mask) - np.min(spoof_mask)) * 255.0

# Convert the masks to uint8
live_mask = live_mask.astype(np.uint8)
spoof_mask = spoof_mask.astype(np.uint8)

# Show the predicted masks
cv2.imshow("Live Mask", live_mask)
cv2.imshow("Spoof Mask", spoof_mask)
cv2.waitKey(0)
cv2.destroyAllWindows()

1/1 [==============================] - 0s 19ms/step
8.598792
4.689618
8.557599
5.6270533
